<a href="https://colab.research.google.com/github/bhuvnk/END2.0/blob/main/Session%204%20-%20RNN%20are%20dead%2C%20and%20their%20renewed%20relevance/END2_Session_4_Re_write_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Fri May 28 14:58:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# T4 Cool, shall increase some epochs and batch size

In [2]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)

LABEL = data.LabelField(dtype = torch.float)

In [3]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [5]:
print(vars(train_data.examples[0]))

{'text': ['Who', 'would', "n't", 'want', 'to', 'go', 'on', 'road', 'trip', 'with', 'Peter', 'Falk', '?', 'That', 'guy', "'s", 'right', 'eye', 'has', 'more', 'character', 'than', 'most', 'actors', 'today', '.', 'This', 'is', 'the', 'kind', 'of', 'funny', 'and', 'touching', 'movie', 'we', 'are', 'all', 'looking', 'for', 'as', 'a', 'counterbalance', 'to', 'all', 'the', 'bombastic', 'special', 'effects', 'bores', '.', 'Women', 'are', 'going', 'to', 'love', 'it', 'for', 'all', 'the', 'wake', '-', 'up', 'romance', 'advice', 'for', 'men', ',', 'and', 'men', 'will', 'love', 'it', 'for', 'its', 'spot', '-', 'on', 'father', '/', 'son', 'character', 'study', '--', 'one', 'great', 'little', 'scene', 'after', 'another', '.', 'And', 'it', 'has', 'just', 'enough', 'of', 'an', 'edge', 'to', 'be', 'a', 'true', 'indie', 'find', '.', 'Obviously', 'this', 'is', 'a', 'labor', 'of', 'love', 'for', 'Paul', 'Reiser', 'who', 'understands', 'what', 'it', "'s", 'like', 'to', 'be', 'both', 'a', 'father', 'and', '

In [6]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [8]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [10]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202995), (',', 192589), ('.', 166220), ('a', 109823), ('and', 109605), ('of', 100631), ('to', 94086), ('is', 76261), ('in', 61475), ('I', 54572), ('it', 53798), ('that', 49288), ('"', 44123), ("'s", 43366), ('this', 42618), ('-', 36817), ('/><br', 35673), ('was', 35263), ('as', 30534), ('with', 29957)]


In [11]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [12]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [13]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch=True, # Let's add some packed_padded_sequence (will cause issue later with lstm input due to lengh order :/ )
    device = device)

In [14]:
print('Train data dims')
for batch in train_iterator:
    print(f'input_text: {batch.text[0].size()}, length: {batch.text[1].size()}')
    print(f'Target: {batch.label.size()}')
    break

Train data dims
input_text: torch.Size([133, 128]), length: torch.Size([128])
Target: torch.Size([128])


In [15]:
print('Validation data dims')
for batch in valid_iterator:
    print(f'input_text: {batch.text[0].size()}, length: {batch.text[1].size()}')
    print(f'Target: {batch.label.size()}')
    break

Validation data dims
input_text: torch.Size([54, 128]), length: torch.Size([128])
Target: torch.Size([128])


In [16]:
print('Test data dims')
for batch in test_iterator:
    print(f'input_text: {batch.text[0].size()}, length: {batch.text[1].size()}')
    print(f'Target: {batch.label.size()}')
    break

Test data dims
input_text: torch.Size([42, 128]), length: torch.Size([128])
Target: torch.Size([128])


## Replacing single RNN to stacked 2 layer LSTM

In [17]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_length):

        #text = [sent len, batch size]
        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]
        
        #Using packed sequence here
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded,
                                                         text_length.to('cpu')) # important to avoid grad errors
        
        #  output: [sentence len, batch size, hidden size]

        #  hidden: [1, batch size, hidden size]

        packed_output, (hidden, cell) = self.lstm1(packed)
        packed_output, (hidden, cell) = self.lstm2(hidden)
        
        return self.fc(hidden.squeeze(0)).view(-1)


In [18]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,122,113 trainable parameters


In [20]:
import torch.optim as optim

# optimizer = optim.SGD(model.parameters(), lr=1e-3)

# Changing optimizer to ADAM with reduced 1/10th of sgd
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [21]:
criterion = nn.BCEWithLogitsLoss()

In [22]:
model = model.to(device)
criterion = criterion.to(device)

In [23]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [24]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:

        # Few changes to adjust for the packed sequence
        text, text_length = batch.text
        logits = model(text, text_length)
        
        loss = criterion(logits, batch.label)
        
        acc = binary_accuracy(logits, batch.label)
        
        optimizer.zero_grad()

        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [25]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            # Few changes to adjust for the packed sequence
            text, text_length = batch.text
            predictions = model(text, text_length)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [26]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [27]:
# Increasing some epochs
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 0.692 | Train Acc: 52.82%
	 Val. Loss: 0.688 |  Val. Acc: 54.76%
Epoch: 02 | Epoch Time: 0m 7s
	Train Loss: 0.652 | Train Acc: 62.62%
	 Val. Loss: 0.611 |  Val. Acc: 67.34%
Epoch: 03 | Epoch Time: 0m 7s
	Train Loss: 0.595 | Train Acc: 68.73%
	 Val. Loss: 0.574 |  Val. Acc: 70.73%
Epoch: 04 | Epoch Time: 0m 7s
	Train Loss: 0.562 | Train Acc: 71.79%
	 Val. Loss: 0.553 |  Val. Acc: 71.83%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 0.537 | Train Acc: 74.19%
	 Val. Loss: 0.584 |  Val. Acc: 69.84%
Epoch: 06 | Epoch Time: 0m 7s
	Train Loss: 0.510 | Train Acc: 75.89%
	 Val. Loss: 0.510 |  Val. Acc: 75.48%
Epoch: 07 | Epoch Time: 0m 7s
	Train Loss: 0.463 | Train Acc: 79.15%
	 Val. Loss: 0.483 |  Val. Acc: 77.35%
Epoch: 08 | Epoch Time: 0m 7s
	Train Loss: 0.433 | Train Acc: 81.09%
	 Val. Loss: 0.461 |  Val. Acc: 78.97%
Epoch: 09 | Epoch Time: 0m 7s
	Train Loss: 0.406 | Train Acc: 82.71%
	 Val. Loss: 0.454 |  Val. Acc: 79.12%
Epoch: 10 | Epoch Time: 0m 7

In [28]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.366 | Test Acc: 85.01%
